<a href="https://colab.research.google.com/github/souradip93/CS69002_9A_18CS60R07/blob/master/Assignment9A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import pandas as pd
import numpy as np
import io
import nltk
import re

In [30]:
from google.colab import files
uploaded = files.upload()

Saving Train_20K.txt to Train_20K.txt


In [31]:
df_train = pd.read_csv(io.StringIO(uploaded['Train_20K.txt'].decode('utf-8')), sep='\t')
df_train.head()

,text,label
0,John Waters has given us a genuinely enjoyable...,1
1,This first two seasons of this comedy series w...,1
2,"What an unfortunate mess is ""Shiner."" I wanted...",0
3,I'm not entirely sure Rob Schmidt qualifies as...,1
4,i wasn't sure whether to laugh or cry. Porrett...,0


In [14]:
df_test = pd.read_csv(io.StringIO(uploaded['Test_5K.txt'].decode('utf-8')), sep='\t')
df_test.head()

,text,label
0,as always this is an inaccurate picture of the...,0
1,Did the movie-makers even preview this before ...,0
2,"Heavily re-edited and often confusing, the ori...",0
3,I notice that most of the people who think thi...,0
4,"First of all, this is a low-budget movie, so m...",0


In [32]:
print('Number of Negative movie reviews', len(df_train[df_train['label']==0]))
print('Number of Positive movie reviews', len(df_train[df_train['label']==1]))

Number of Negative movie reviews 8994
Number of Positive movie reviews 9005


In [0]:
def remove_br_tags_and_numbers(train_data_X):
  import re
  regex = re.compile(r'<br.*?>|\d+')
  train_data_X = [regex.sub('', x) for x in train_data_X]
  return train_data_X

In [0]:

def tokenize(train_data_X):
  from nltk.tokenize import RegexpTokenizer
  tokenizer = RegexpTokenizer(r'\w+')
  train_data_X = [tokenizer.tokenize(x) for x in train_data_X]
  #train_data_X[0], train_data_Y[0]
  return train_data_X


In [0]:
def convert_to_lower(train_data_X):
  train_data_X = [[y.lower() for y in x] for x in train_data_X]
  #train_data_X[0], train_data_Y[0]
  return train_data_X

In [0]:
def remove_stopwords(train_data_X):
  from nltk.corpus import stopwords
  nltk.download('stopwords')
  stopword_set = set(stopwords.words('english'))
  train_data_X = [[y for y in x if y not in stopword_set] for x in train_data_X]
  #train_data_X[0], train_data_Y[0]
  return train_data_X

In [0]:
def lemmatize(train_data_X):
  from nltk.stem import WordNetLemmatizer 
  nltk.download('wordnet')
  lemmatizer = WordNetLemmatizer()
  train_data_X = [[lemmatizer.lemmatize(y) for y in x] for x in train_data_X]
  return train_data_X

In [0]:
def generate_word_ids(dataset):
  word_to_ix = {}
  for sent in dataset:
      for word in sent:
          if word not in word_to_ix:
              word_to_ix[word] = len(word_to_ix)
              
  word_to_ix['UNKNOWN'] = len(word_to_ix)
  return word_to_ix

In [0]:
train_data_X = df_train['text'].astype(str).tolist()
train_data_Y = df_train['label'].astype(int)

In [0]:
test_data_X = df_test['text'].astype(str).tolist()
test_data_Y = df_test['label'].astype(int)

In [43]:
train_data_X = remove_br_tags_and_numbers(train_data_X)
train_data_X = tokenize(train_data_X)
train_data_X = convert_to_lower(train_data_X)
train_data_X = remove_stopwords(train_data_X)
train_data_X = lemmatize(train_data_X)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [44]:
test_data_X = remove_br_tags_and_numbers(test_data_X)
test_data_X = tokenize(test_data_X)
test_data_X = convert_to_lower(test_data_X)
test_data_X = remove_stopwords(test_data_X)
test_data_X = lemmatize(test_data_X)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [45]:
word_to_ix = generate_word_ids(train_data_X + test_data_X)

VOCAB_SIZE = len(word_to_ix)
NUM_LABELS = 2
print(VOCAB_SIZE, word_to_ix)

64610 {'john': 0, 'water': 1, 'given': 2, 'u': 3, 'genuinely': 4, 'enjoyable': 5, 'film': 6, 'certainly': 7, 'without': 8, 'shocking': 9, 'esque': 10, 'moment': 11, 'tamer': 12, 'older': 13, 'culty': 14, 'stuff': 15, 'pink': 16, 'flamingo': 17, 'pecker': 18, 'harkens': 19, 'back': 20, 'early': 21, 'mainstream': 22, 'stage': 23, 'reminds': 24, 'viewer': 25, 'kind': 26, 'humor': 27, 'evident': 28, 'polyester': 29, 'overall': 30, 'really': 31, 'fun': 32, 'comedy': 33, 'great': 34, 'first': 35, 'two': 36, 'season': 37, 'series': 38, 'strange': 39, 'funny': 40, 'drama': 41, 'element': 42, 'bill': 43, 'mother': 44, 'struggling': 45, 'usual': 46, 'problem': 47, 'life': 48, 'bit': 49, 'depressing': 50, 'mix': 51, 'well': 52, 'th': 53, 'probably': 54, 'dropped': 55, 'soon': 56, 'became': 57, 'one': 58, 'funniest': 59, 'bbc': 60, 'ever': 61, 'made': 62, 'chemistry': 63, 'ben': 64, 'character': 65, 'always': 66, 'many': 67, 'brilliant': 68, 'memorable': 69, 'sketch': 70, 'christmas': 71, 'special

In [46]:
label_to_ix = {1: 1, 0: 0}
ix_to_label = {v: k for k, v in label_to_ix.items()}

label_to_ix, ix_to_label

({0: 0, 1: 1}, {0: 0, 1: 1})

In [47]:
import torch.nn as nn
import torch.nn.functional as F

SEED = 42

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.device('cuda')



device(type='cuda')

In [0]:
class BOWClassifier(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(BOWClassifier, self).__init__()
    self.lin1 = nn.Linear(vocab_size, 50)
    self.lin2 = nn.Linear(50, num_labels)
    
  def forward(self, x):
    out = self.lin1(x)
    return F.softmax(self.lin2(out))

In [0]:
def make_bow_vector(sentence, word_to_ix):
    # create a vector of zeros of vocab size = len(word_to_idx)
    vec = torch.zeros(len(word_to_ix))
    for word in sentence:
        if word not in word_to_ix:
            #raise ValueError('Word',word,' not present in the dictionary. Sorry!')
            vec[ word_to_ix['UNKNOWN'] ] += 1
        else:
            vec[word_to_ix[word]]+=1
    return vec.view(1, -1)

def make_target(label, label_to_ix):
    #print(label)
    return torch.LongTensor([label_to_ix[label]])

In [0]:
bow = BOWClassifier(NUM_LABELS, VOCAB_SIZE).cuda()

In [0]:
# define a loss function and an optimizer
loss_function = nn.NLLLoss()
opt = torch.optim.SGD(bow.parameters(), lr = 0.01)

In [0]:
from torch.autograd import Variable
import time

bow_vec = []
for instance in train_data_X:
  ins_bow_vector = make_bow_vector(instance, word_to_ix)
  bow_vec.append(Variable(ins_bow_vector).cuda())
  
label = []
for l in train_data_Y:
  label_vector = make_target(l, label_to_ix)
  label.append(Variable(label_vector).cuda())



In [79]:
# the training loop
for epoch in range(20):
    start = time.time()
    for ind, instance in enumerate(train_data_X):
        bow.zero_grad()
        probs = bow(bow_vec[ind]) # forward pass
        loss = loss_function(probs, label[ind])
        loss.backward()
        opt.step()
    end = time.time()
    print('Time Taken - ' + str(end - start))
    print('Epoch - ' + str(epoch) + ' , LOSS - ' + str(loss.data))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':


Time Taken - 28.068181037902832
Epoch - 0 , LOSS - tensor(0., device='cuda:0')
Time Taken - 28.84271001815796
Epoch - 1 , LOSS - tensor(0., device='cuda:0')
Time Taken - 28.225021839141846
Epoch - 2 , LOSS - tensor(0., device='cuda:0')
Time Taken - 28.04674458503723
Epoch - 3 , LOSS - tensor(-3.4508e-39, device='cuda:0')
Time Taken - 28.074090719223022
Epoch - 4 , LOSS - tensor(-1.1785e-38, device='cuda:0')
Time Taken - 28.856481075286865
Epoch - 5 , LOSS - tensor(-2.1176e-34, device='cuda:0')
Time Taken - 28.243013620376587
Epoch - 6 , LOSS - tensor(-6.0516e-31, device='cuda:0')
Time Taken - 28.07004141807556
Epoch - 7 , LOSS - tensor(-9.1524e-20, device='cuda:0')
Time Taken - 28.075011730194092
Epoch - 8 , LOSS - tensor(-2.2232e-39, device='cuda:0')
Time Taken - 28.939279079437256
Epoch - 9 , LOSS - tensor(-8.3314e-23, device='cuda:0')
Time Taken - 28.25094985961914
Epoch - 10 , LOSS - tensor(-5.1868e-15, device='cuda:0')
Time Taken - 28.016115188598633
Epoch - 11 , LOSS - tensor(-1.

In [54]:
import torch
torch.save(bow,'model.json')

from google.colab import files
model = files.download("model.json")

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type BOWClassifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [0]:
import pickle

pickle_out = open("word_to_ix.pickle","wb")
pickle.dump(word_to_ix, pickle_out)
pickle_out.close()

from google.colab import files
model = files.download("word_to_ix.pickle")


In [57]:
from google.colab import files
temp_test = files.upload()


Saving model.json to model (1).json


In [0]:
import torch
bow = torch.load(io.BytesIO(temp_test['model.json']))

In [59]:
from google.colab import files
temp_test = files.upload()

Saving word_to_ix.pickle to word_to_ix (1).pickle


In [0]:
pickle_in = open("word_to_ix.pickle","rb")
word_to_ix = pickle.load(pickle_in)

In [80]:
print('--- AFTER TRAINING ---')
count = 0
for ind, instance in enumerate(test_data_X):
    bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
    logprobs = bow(bow_vec)
    pred = np.argmax(logprobs.cpu().data.numpy())
    label = test_data_Y[ind]
    if label == ix_to_label[pred]:
      count += 1
      
print(float(count)/ len(test_data_X))

--- AFTER TRAINING ---


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':


0.835
